# Aula 08 - Dados Desbalanceados

Nosso objetivo nessa aula é apresentar algumas ferramentas úteis para lidar com dados desbalanceados, ou seja, quando temos muitos dados de uma classe e poucos dados de outra. Note que este assunto será muito útil nas disciplinas que ainda estão por vir, especialmente no contexto de aprendizado supervisionado!

Em muitos casos, há um desequilíbrio nas classes a serem analisadas, com uma classe tendo muito mais amostras do que a outra. Por exemplo, requisições de uso de seguro legítimas versus fraudulentas, ou navegadores versus compradores em um site. A classe rara (por exemplo, as requisições fraudulentas) é geralmente a classe de mais interesse e é normalmente designada com a flag 1, em contraste com os 0s (mais prevalentes).

No cenário típico, os 1s são o caso mais importante, no sentido de que classificá-los incorretamente como 0s é mais caro do que classificar 0s como 1s. Por exemplo, identificar corretamente uma reclamação de seguro fraudulenta pode economizar milhares de reais. Por outro lado, identificar corretamente uma reclamação não fraudulenta simplesmente economiza o custo e o esforço de processar a reclamação manualmente com uma revisão mas cuidadosa (que é o que você faria se a reclamação fosse marcada como “fraudulenta”).

Nesses casos, a menos que as classes sejam facilmente separáveis, a abordagem mais simples para um classificador seria utilizar um modelo que simplesmente classifica tudo como 0. Por exemplo, se apenas 0,1% dos visitantes de uma loja da web acabassem comprando, um modelo que prevê que todos os visitantes sairão sem comprar nada terá 99,9% de precisão. Apesar disso, esse modelo será inútil.

Em vez disso, seria muito mais útil um modelo com precisão geral menor, mas que fosse bom em detectar os visitantes compradores (mesmo que classifique erroneamente alguns não-compradores ao longo do caminho).

Para isso, há outras métricas úteis (que serão abordadas em outras disciplinas), mas aqui, hoje, iremos apresentar algumas formas de tentar equilibrar o dataset.

Utilizaremos um dataset de empréstimos bancários para realizar nossas análises (full_train_set.csv).

## Undersampling (sub-amostragem)

Se tivermos dados suficientes, como é o caso do nosso dataset de empréstimos, uma solução é subamostrar (ou reduzir) a classe predominante para que os dados a serem aprendidos estejam equilibrados.

A ideia básica na subamostragem é que a classe predominante possui muitos registros redundantes e, por isso, podemos descartá-los. Lidar com uma base de dados menor e mais balanceada nos permite alcançar benefícios de performance e torna mais fácil o preparo dos dados.

Surge, então, uma pergunta: qual a quantidade de dados que pode ser considerada como suficiente para fazer uma análise ou treinar um modelo? Essa pergunta não possui uma resposta objetiva, pois depende da aplicação. Em geral, dezenas de milhares de registros para a classe rara são suficientes. Quanto mais distinguíveis as classes forem, de menos dados precisaremos.

Na nossa base de dados, apenas cerca de 19% dos empréstimos estão inadimplentes, conforme veremos a seguir.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('full_train_set.csv')

In [3]:
df.head() # coluna outcome

,status,loan_amnt,term,annual_inc,dti,payment_inc_ratio,revol_bal,revol_util,purpose,home_ownership,delinq_2yrs_zero,pub_rec_zero,open_acc,grade,outcome,emp_length,purpose_,home_,emp_len_
0,Fully Paid,5000,36 months,24000,27.65,8.14350,13648.0,83.7,credit_card,RENT,1,1,3,5.4,paid off,11,credit_card,RENT,> 1 Year
1,Charged Off,2500,60 months,30000,1.00,2.39320,1687.0,9.4,car,RENT,1,1,3,4.8,default,1,major_purchase,RENT,> 1 Year
2,Fully Paid,2400,36 months,12252,8.72,8.25955,2956.0,98.5,small_business,RENT,1,1,2,5.0,paid off,11,small_business,RENT,> 1 Year
3,Fully Paid,10000,36 months,49200,20.00,8.27585,5598.0,21.0,other,RENT,1,1,10,4.2,paid off,11,other,RENT,> 1 Year
4,Fully Paid,5000,36 months,36000,11.20,5.21533,7963.0,28.3,wedding,RENT,1,1,9,6.8,paid off,4,other,RENT,> 1 Year


In [4]:
df.shape

(119987, 19)

In [5]:
df['outcome'].unique()

array(['paid off', 'default'], dtype=object)

In [6]:
total = df.shape[0]

counts = df['outcome'].value_counts()

paid_off = counts[0]
default = counts[1]

print(f'Paid_off: {(paid_off/total)*100}% => {paid_off}')
print(f'Default: {(default/total)*100}% => {default}')

Paid_off: 81.1054530907515% => 97316
Default: 18.894546909248504% => 22671


Para que possamos avaliar o efeito de uma base de dados desbalanceada, iremos utilizar a Regressão Logística como uma caixa preta.

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression

predictors = ['payment_inc_ratio', 'purpose_', 'home_', 'emp_len_', 'dti', 'revol_bal', 'revol_util']
outcome = 'outcome'

X = pd.get_dummies(df[predictors], prefix='', prefix_sep='', drop_first=True)
y = df[outcome]

model = LogisticRegression(penalty='l2', C=1e42, solver='liblinear')
model.fit(X, y)

print(f'Porcentagem de empréstimos preditos como inadimplentes: {100*np.mean(model.predict(X) == "default")}%')

Porcentagem de empréstimos preditos como inadimplentes: 0.15585021710685323%


Apenas 0,15% dos empréstimos são preditos como inadimplentes. Note que, na verdade, cerca de 19% são inadimplentes na base de dados!

Os empréstimos que foram pagos superam os empréstimos inadimplentes porque o modelo é treinado usando todos os dados igualmente. Intuitivamente, a presença de tantos empréstimos pagos significa que, mesmo para um empréstimo inadimplente, o modelo provavelmente encontrará alguns empréstimos pagos que possuem características semelhantes a ele.

Vamos realizar um rápido balanceamento por subamostragem e re-executar nosso classificador.

In [8]:
default_df = df[df['outcome'] == 'default']

paid_off_df = df[df['outcome'] == 'paid off']

paid_off_df = paid_off_df.iloc[:default]

paid_off_df.shape

(22671, 19)

Agora sim temos a mesma quantidade de entradas para as duas classes.

In [9]:
undersample_df = pd.concat([paid_off_df, default_df])

total = undersample_df.shape[0]

counts = undersample_df['outcome'].value_counts()

paid_off = counts[0]
default = counts[1]

print(f'Paid_off: {(paid_off/total)*100}% => {paid_off}')
print(f'Default: {(default/total)*100}% => {default}')

Paid_off: 50.0% => 22671
Default: 50.0% => 22671


Vamos re-executar o preditor.

In [10]:
X = pd.get_dummies(undersample_df[predictors], prefix='', prefix_sep='', drop_first=True)
y = undersample_df[outcome]

model = LogisticRegression(penalty='l2', C=1e42, solver='liblinear')
model.fit(X, y)

print(f'Porcentagem de empréstimos preditos como inadimplentes: {100*np.mean(model.predict(X) == "default")}%')

Porcentagem de empréstimos preditos como inadimplentes: 50.45432490847338%


Note que, agora, temos cerca de 50% dos dados preditos como inadimplentes.

## Oversampling e Up/Down Weighting

O principal problema do uso de undersampling é que ele descarta dados que poderiam ser utilizados para treinar o modelo. Se tivermos um dataset pequeno e a classe rara possui apenas algumas centenas ou poucos milhares de amostras, subamostrar a classe predominante pode resultar na perda de informações muito úteis!

Nesse caso, iremos superamostrar (oversampling) a classe rara por meio de uma seleção aleatória de linhas com reposição (bootstrapping), ou seja, serão selecionadas, aleatoriamente, algumas linhas da classe rara para serem repetidas (com reposição) de tal forma que tenhamos, ao final da seleção, a mesma quantidade de elementos para as duas classes.

Podemos alcançar um efeito similar através da pesagem dos dados. Muitos algoritmos de classificação podem receber um argumento com o peso que permitirá aumentar ou diminuir o peso dos dados.

A maioria dos métodos do scikit-learn permite especificar pesos pela função **fit** utilizando o argumento **sample_weight**.

In [11]:
default_wt = 1 / np.mean(df['outcome'] == 'default')
wt = [default_wt if outcome == 'default' else 1 for outcome in df['outcome']]

X = pd.get_dummies(df[predictors], prefix='', prefix_sep='', drop_first=True)
y = df[outcome]

model = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
model.fit(X, y, wt)

print(f'Porcentagem de empréstimos preditos como inadimplentes (weighting): {100*np.mean(model.predict(X) == "default")}%')

Porcentagem de empréstimos preditos como inadimplentes (weighting): 61.45999149907907%


Os pesos para os empréstimos inadimplentes são setados para $\frac{1}{p}$, onde $p$ é a probabilidade de inadimplência.

>$\frac{1}{p} \times quantidade\_inadimplencias = 1 \times quantidade\_pagos$

Os empréstimos pagos possuem peso 1. Note que a soma dos pesos para os empréstimos pagos e dos inadimplentes são iguais.

Agora, obtivemos cerca de 61% das predições como sendo inadimplentes (contra os 0,15% obtidos com a base desbalanceada).

Apresentamos, aqui, duas abordagens: definição de pesos (implementada em código) e a superamostragem (ou oversampling, cuja implementação fica como exercício para o leitor).

## Geração de Dados

Uma variação do upsampling via bootstrapping é a geração de dados por meio da uma variação nos dados existentes para criar novos dados.

A intuição dessa técnica é a seguinte: já que possuímos um conjunto limitado de dados, o algoritmo classificador não tem acesso a um conjunto de dados rico o suficiente para construir "regras" de classificação. Sendo assim, ao criar novos dados que são similares (mas não idênticos) aos originais, o algoritmo tem a chance de aprender "regras" de classificação mais robustas.

Essa abordagem ganhou mais atenção com a publicação do algoritmo SMOTE, que significa "Synthetic Minority Oversampling Technique". Esse algoritmo encontra uma entrada que é similar à entrada a ser superamostrada e cria uma nova entrada sintética que é uma média aleatoriamente ponderada entre a entrada original e a entrada vizinha, onde os pesos são gerados separadamente para cada preditor. A quantidade de novas amostras geradas é determinada pela quantidade necessária de novos dados para que o dataset fique balanceado.

O pacote **imbalanced-learn** do Python implementa uma variedade de métodos com uma API compatível com o **scikit-learn**, provendo métodos para super e sub amostragem.

A célula abaixo avalia dois algoritmos de geração de dados: [SMOTE](http://glemaitre.github.io/imbalanced-learn/generated/imblearn.over_sampling.SMOTE.html) e [ADASYN](http://glemaitre.github.io/imbalanced-learn/generated/imblearn.over_sampling.ADASYN.html). Detalhes sobre as implementações desses algoritmos ficam como exercício para o leitor.

In [12]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE

print(f'===== Porcentagem de empréstimos inadimplentes antes da geração de dados: {100*np.mean(y == "default")} =====\n')

# SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X, y)
print(f'Porcentagem de empréstimos inadimplentes após geração de dados via SMOTE: {100*np.mean(y_resampled == "default")}')

model = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
model.fit(X_resampled, y_resampled)
print(f'Porcentagem de empréstimos inadimplentes previstos pela Regressão Logística com dados SMOTE: {100*np.mean(model.predict(X) == "default")}\n')

# ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(X, y)
print(f'Porcentagem de empréstimos inadimplentes após geração de dados via ADASYN: {100*np.mean(y_resampled == "default")}')

model = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
model.fit(X_resampled, y_resampled)
print(f'Porcentagem de empréstimos inadimplentes previstos pela Regressão Logística com dados ADASYN: {100*np.mean(model.predict(X) == "default")}')

===== Porcentagem de empréstimos inadimplentes antes da geração de dados: 18.894546909248504 =====

Porcentagem de empréstimos inadimplentes após geração de dados via SMOTE: 50.0
Porcentagem de empréstimos inadimplentes previstos pela Regressão Logística com dados SMOTE: 29.46152499854151

Porcentagem de empréstimos inadimplentes após geração de dados via ADASYN: 48.56040383751355
Porcentagem de empréstimos inadimplentes previstos pela Regressão Logística com dados ADASYN: 27.543817246868414


## Exercícios

Utilizando o dataset de fraudes em cartões de crédito, aplique as diferentes técnicas para dados desbalanceados e compare, utilizando a Regressão Logística, os resultados obtidos na predição de fraudes.

Qual técnica obteve o melhor desempenho na detecção de fraudes?

In [13]:
cc_df = pd.read_csv('dataset/creditcard.csv')

cc_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [14]:
total = cc_df.shape[0]

counts = cc_df['Class'].value_counts()

not_fraud = counts[0]
fraud = counts[1]

print(f'Not fraud: {(not_fraud/total)*100}% => {not_fraud}')
print(f'Fraud: {(fraud/total)*100}% => {fraud}')

Not fraud: 99.82725143693798% => 284315
Fraud: 0.1727485630620034% => 492
